<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-13 13:10:06
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.38 C
-------------------
Today PnL: -1.66 L (-1.19%)
Current PnL: -31.94 L (-20.37%)
CY Booked + Current PnL: -17.20 L (-10.97%)
-------------------
Total profit:  1.37 L
Total loss:  -33.31 L
-------------------
Total Booked + Current PnL: 9.78 L (7.53%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.72%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 97.30 L (70.74%)
Deployed:  1.30 C
Current:  1.38 C
CAGR/XIRR %: 2.98%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.95,3.87,17.03,21.57,17332.0,3795.0,101775.0,397.04,-10.57,62.0,H-SC,3.71,168.0,0.22,0.74,48.52,XR,ATH,FINANCE
77,TTKPRESTIG,-2.47,-26.79,36.72,0.09,27092.0,-26998.0,73779.0,770.00,71.18,40.0,M-SC,8.78,253.0,-1.00,0.54,1.54,OX40N,NTT,DURABLES
18,COALINDIA,-2.54,0.45,18.69,19.23,28249.0,684.0,151145.0,484.83,18.42,41.0,L-LC,11.77,182.0,0.02,1.10,21.83,XY25,ATH,MINING
36,ICICIGI,-0.18,1.51,17.57,19.34,35690.0,3019.0,203128.0,2252.93,-19.79,57.0,X-MC,3.27,71.0,0.08,1.48,17.69,X40,ATH,INSURANCE
51,MEDANTA,-0.91,-4.96,30.56,24.08,36870.0,-6301.0,120649.0,1486.00,-12.71,52.0,X-SC,11.72,91.0,-0.17,0.88,10.50,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,6.43,-43.22,138.24,35.27,89129.0,-49075.0,64474.0,80.00,-55.73,56.0,L-SC,14.33,268.0,-0.55,0.47,57.80,XR,NTT,HOTELS
50,MASFIN,0.95,3.87,17.03,21.57,17332.0,3795.0,101775.0,397.04,-10.57,62.0,H-SC,3.71,168.0,0.22,0.74,48.52,XR,ATH,FINANCE
83,WHIRLPOOL,0.92,-28.15,153.19,81.92,126356.0,-32315.0,82483.0,2270.00,-57.90,60.0,M-SC,30.81,218.0,-0.26,0.60,15.86,XR,NTT,DURABLES
76,TRIDENT,0.52,-28.09,78.70,28.51,52213.0,-25910.0,66344.0,48.00,-14.99,50.0,M-SC,8.40,226.0,-0.50,0.48,13.52,XR,NTT,TEXTILES
1,ABB,0.38,6.61,36.55,45.57,101939.0,17285.0,278904.0,7934.00,-31.49,69.0,H-MC,2.96,121.0,0.17,2.03,23.85,AR,NTT,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,-5.53,-25.70,102.93,50.79,107574.0,-36141.0,104512.0,831.7,-4.52,44.0,H-SC,7.51,171.0,-0.34,0.76,21.69,XR,ATH,IT
70,SYMPHONY,-5.28,-32.26,47.57,-0.03,55150.0,-55206.0,115935.0,1306.0,-45.51,45.0,M-SC,10.37,206.0,-1.00,0.84,7.14,OX40N,NTT,DURABLES
34,HINDUNILVR,-3.69,-9.71,25.90,13.67,60111.0,-24973.0,232090.0,2922.0,-16.44,41.0,X-LC,2.71,9.0,-0.42,1.69,9.57,XY25,NTT,FMCG
64,SHALBY,-3.55,-37.87,101.37,25.10,114078.0,-68607.0,112536.0,327.0,727.75,36.0,M-SC,16.04,248.0,-0.60,0.82,0.00,XY24,NTT,HEALTHCARE
55,RAJESHEXPO,-3.06,-65.42,189.40,0.07,89836.0,-89745.0,47432.0,518.0,1638.64,52.0,L-SC,8.82,269.0,-1.00,0.34,17.97,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-2.54,0.45,18.69,19.23,28249.0,684.0,151145.0,484.83,18.42,41.0,L-LC,11.77,182.0,0.02,1.1,21.83,XY25,ATH,MINING


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.08,3.83,100.25,107.91,153676.0,5649.0,153293.0,1641.55,-15.69,68.0,M-SC,9.93,220.0,0.04,1.11,15.33,OX40N,ATH,CABLES
66,SIS,-2.75,-23.98,59.52,21.26,50238.0,-26627.0,84405.0,528.00,1991.71,49.0,H-SC,4.03,163.0,-0.53,0.61,16.36,OX40N,NTT,MISC
77,TTKPRESTIG,-2.47,-26.79,36.72,0.09,27092.0,-26998.0,73779.0,770.00,71.18,40.0,M-SC,8.78,253.0,-1.00,0.54,1.54,OX40N,NTT,DURABLES
72,TATAELXSI,-2.58,-30.14,88.98,32.03,81952.0,-39733.0,92102.0,9161.00,-24.43,32.0,H-SC,6.18,157.0,-0.48,0.67,3.16,OX40N,NTT,IT
40,INDIGOPNTS,-2.64,-31.02,44.99,0.02,54175.0,-54143.0,120416.0,1408.00,83.59,27.0,M-SC,7.45,240.0,-1.00,0.88,4.74,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-2.54,0.45,18.69,19.23,28249.0,684.0,151145.0,484.83,18.42,41.0,L-LC,11.77,182.0,0.02,1.10,21.83,XY25,ATH,MINING
50,MASFIN,0.95,3.87,17.03,21.57,17332.0,3795.0,101775.0,397.04,-10.57,62.0,H-SC,3.71,168.0,0.22,0.74,48.52,XR,ATH,FINANCE
26,FINCABLES,0.08,3.83,100.25,107.91,153676.0,5649.0,153293.0,1641.55,-15.69,68.0,M-SC,9.93,220.0,0.04,1.11,15.33,OX40N,ATH,CABLES
1,ABB,0.38,6.61,36.55,45.57,101939.0,17285.0,278904.0,7934.00,-31.49,69.0,H-MC,2.96,121.0,0.17,2.03,23.85,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.95,3.87,17.03,21.57,17332.0,3795.0,101775.0,397.04,-10.57,62.0,H-SC,3.71,168.0,0.22,0.74,48.52,XR,ATH,FINANCE
1,ABB,0.38,6.61,36.55,45.57,101939.0,17285.0,278904.0,7934.00,-31.49,69.0,H-MC,2.96,121.0,0.17,2.03,23.85,AR,NTT,ELECTRICAL
26,FINCABLES,0.08,3.83,100.25,107.91,153676.0,5649.0,153293.0,1641.55,-15.69,68.0,M-SC,9.93,220.0,0.04,1.11,15.33,OX40N,ATH,CABLES
39,INDIAMART,0.12,-5.34,118.39,106.72,138218.0,-6588.0,116748.0,4810.62,-53.11,47.0,H-SC,7.62,138.0,-0.05,0.85,16.79,AR,ATH,MISC
85,ZYDUSLIFE,-1.30,-4.15,41.50,35.63,83364.0,-8696.0,200878.0,1286.17,-17.63,53.0,H-MC,5.94,119.0,-0.10,1.46,11.83,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-2.21,-7.74,45.50,34.24,131349.0,-24209.0,288679.0,1972.00,-28.21,19.0,X-LC,4.77,6.0,-0.18,2.10,0.00,X40,NTT,IT
33,HCLTECH,-1.04,-5.20,30.63,23.83,70249.0,-12578.0,229346.0,1908.19,-0.81,22.0,X-LC,5.32,13.0,-0.18,1.67,10.03,X40,ATH,IT
8,AWL,-2.09,-34.53,140.11,57.19,299139.0,-112624.0,213503.0,485.00,-69.12,25.0,X-MC,17.41,58.0,-0.38,1.55,0.00,XY24,NTT,FMCG
73,TCS,-2.43,-26.22,60.68,18.55,169342.0,-99163.0,279074.0,4311.59,-36.57,25.0,X-LC,2.41,3.0,-0.59,2.03,0.00,X40,ATH,IT
13,BERGEPAINT,-0.69,-18.49,48.62,21.14,90097.0,-42031.0,185308.0,680.00,-24.20,27.0,X-MC,11.09,75.0,-0.47,1.35,0.00,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,-0.68,0.88,42.44,43.70,106210.0,2192.0,250260.0,735.00,-7.41,54.0,X-MC,1.83,73.0,0.02,1.82,16.07,XY24,BTT,FMCG
81,VBL,-0.26,-7.95,47.39,35.67,137780.0,-25105.0,290737.0,671.64,-18.89,42.0,X-LC,2.00,5.0,-0.18,2.11,4.80,X40N,ATH,FMCG
73,TCS,-2.43,-26.22,60.68,18.55,169342.0,-99163.0,279074.0,4311.59,-36.57,25.0,X-LC,2.41,3.0,-0.59,2.03,0.00,X40,ATH,IT
78,UNITDSPR,-0.39,3.71,16.45,20.77,39946.0,8686.0,242830.0,1644.00,-5.34,62.0,X-MC,2.56,70.0,0.22,1.77,11.39,X40N,NTT,BREWERIES
34,HINDUNILVR,-3.69,-9.71,25.90,13.67,60111.0,-24973.0,232090.0,2922.00,-16.44,41.0,X-LC,2.71,9.0,-0.42,1.69,9.57,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-0.71,-11.20,32.96,18.07,52347.0,-20035.0,158820.0,35195.00,-23.91,35.0,X-MC,10.13,64.0,-0.38,1.15,0.0,X40,ATH,PHARMA
8,AWL,-2.09,-34.53,140.11,57.19,299139.0,-112624.0,213503.0,485.00,-69.12,25.0,X-MC,17.41,58.0,-0.38,1.55,0.0,XY24,NTT,FMCG
13,BERGEPAINT,-0.69,-18.49,48.62,21.14,90097.0,-42031.0,185308.0,680.00,-24.20,27.0,X-MC,11.09,75.0,-0.47,1.35,0.0,XY24,NTT,PAINTS
42,INFY,-2.21,-7.74,45.50,34.24,131349.0,-24209.0,288679.0,1972.00,-28.21,19.0,X-LC,4.77,6.0,-0.18,2.10,0.0,X40,NTT,IT
73,TCS,-2.43,-26.22,60.68,18.55,169342.0,-99163.0,279074.0,4311.59,-36.57,25.0,X-LC,2.41,3.0,-0.59,2.03,0.0,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.20,-30.60,105.83,42.84,47744.0,-19892.0,45114.0,424.00,-54.82,50.0,X-SC,19.99,83.0,-0.42,0.33,8.52,XY24,NTT,MISC
58,RELAXO,-2.21,-53.57,228.03,52.31,153688.0,-77762.0,67398.0,1176.00,-50.54,35.0,X-SC,13.85,92.0,-0.51,0.49,0.22,X40N,NTT,FOOTWEAR
12,BATAINDIA,-2.89,-44.63,144.20,35.20,102727.0,-57431.0,71239.0,2096.00,-6.57,40.0,X-SC,21.02,93.0,-0.56,0.52,1.24,X40,NTT,FOOTWEAR
51,MEDANTA,-0.91,-4.96,30.56,24.08,36870.0,-6301.0,120649.0,1486.00,-12.71,52.0,X-SC,11.72,91.0,-0.17,0.88,10.50,XY24,NTT,HEALTHCARE
35,HONAUT,-1.47,-22.90,85.88,43.32,107848.0,-37292.0,125580.0,58357.33,-33.73,38.0,X-SC,10.28,90.0,-0.35,0.91,1.96,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.71,-22.75,43.40,10.78,80573.0,-54665.0,185653.0,452.00,-53.82,38.0,X-LC,18.94,1.0,-0.68,1.35,2.26,X40,NTT,FMCG
75,TMPV,0.12,-21.32,56.29,22.98,100053.0,-48152.0,177746.0,600.00,-18.19,67.0,X-LC,3.34,2.0,-0.48,1.29,13.63,XY24,NTT,AUTO
73,TCS,-2.43,-26.22,60.68,18.55,169342.0,-99163.0,279074.0,4311.59,-36.57,25.0,X-LC,2.41,3.0,-0.59,2.03,0.00,X40,ATH,IT
81,VBL,-0.26,-7.95,47.39,35.67,137780.0,-25105.0,290737.0,671.64,-18.89,42.0,X-LC,2.00,5.0,-0.18,2.11,4.80,X40N,ATH,FMCG
42,INFY,-2.21,-7.74,45.50,34.24,131349.0,-24209.0,288679.0,1972.00,-28.21,19.0,X-LC,4.77,6.0,-0.18,2.10,0.00,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.43,-32.17,95.93,32.89,50235.0,-24840.0,52366.0,1800.00,-746.53,55.0,L-MC,11.15,259.0,-0.49,0.38,44.21,XR,NTT,BANKS
67,SONACOMS,-1.14,-8.88,52.65,39.09,48529.0,-8986.0,92173.0,804.02,-28.76,62.0,M-SC,4.42,227.0,-0.19,0.67,30.39,AR,ATH,AUTO
50,MASFIN,0.95,3.87,17.03,21.57,17332.0,3795.0,101775.0,397.04,-10.57,62.0,H-SC,3.71,168.0,0.22,0.74,48.52,XR,ATH,FINANCE
79,VAIBHAVGBL,0.13,-20.63,102.56,60.77,148780.0,-37709.0,145066.0,521.00,67.49,62.0,H-SC,1.36,158.0,-0.25,1.05,31.52,XR,NTT,JEWELLERY
18,COALINDIA,-2.54,0.45,18.69,19.23,28249.0,684.0,151145.0,484.83,18.42,41.0,L-LC,11.77,182.0,0.02,1.10,21.83,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,-0.67,-12.25,65.88,45.57,113967.0,-24140.0,172992.0,844.00,3754.05,70.0,M-SC,5.73,245.0,-0.21,1.26,30.91,XY24,NTT,AUTO
1,ABB,0.38,6.61,36.55,45.57,101939.0,17285.0,278904.0,7934.00,-31.49,69.0,H-MC,2.96,121.0,0.17,2.03,23.85,AR,NTT,ELECTRICAL
79,VAIBHAVGBL,0.13,-20.63,102.56,60.77,148780.0,-37709.0,145066.0,521.00,67.49,62.0,H-SC,1.36,158.0,-0.25,1.05,31.52,XR,NTT,JEWELLERY
11,BANDHANBNK,-1.70,-19.83,140.02,92.42,312680.0,-55248.0,223311.0,400.00,57.33,68.0,H-SC,8.08,169.0,-0.18,1.62,29.06,XY24,NTT,BANKS
67,SONACOMS,-1.14,-8.88,52.65,39.09,48529.0,-8986.0,92173.0,804.02,-28.76,62.0,M-SC,4.42,227.0,-0.19,0.67,30.39,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.25
1,25,44.63
2,50,76.65


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.66
MC    31.19
LC    23.15
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.74
X40      23.07
X40N     15.29
AR        9.02
XR        8.89
XY25      8.72
OX40N     7.39
SR        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.45
H-SC    22.87
X-LC    19.83
M-SC    11.59
X-SC    10.39
H-MC     4.97
M-MC     1.39
H-LC     1.14
L-LC     1.10
M-LC     1.08
L-SC     0.81
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.00,-14.71,53.26
FINANCE,13.47,-12.82,57.21
IT,11.53,-34.85,102.13
MISC,7.11,-32.84,87.19
ELECTRICAL,6.22,-9.55,49.52
PAINTS,5.05,-33.19,52.98
INSURANCE,4.87,-1.36,36.04
PHARMA,4.14,-3.06,35.50
AUTO,3.22,-18.35,59.28


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3295331.0,21
AR,1332067.0,10
X40,1325040.0,15
XR,1265793.0,12
X40N,1018198.0,10
OX40N,773954.0,10
XY25,421824.0,6
SR,298056.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3646123.0,24
X-MC,1664791.0,16
M-SC,1464696.0,15
X-LC,1165368.0,12
X-SC,862368.0,9
H-MC,395613.0,3
L-SC,178965.0,2
M-LC,142976.0,1
H-LC,76146.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1251951.0      6
           AR         947609.0      5
M-SC       XY24       822623.0      6
H-SC       XR         804884.0      7
X-LC       X40        606755.0      6
X-MC       X40        544902.0      7
           XY24       495446.0      3
           X40N       413772.0      4
X-SC       X40N       378149.0      4
H-SC       OX40N      343623.0      4
M-SC       OX40N      340495.0      5
X-SC       XY24       310836.0      3
H-SC       SR         298056.0      2
X-LC       X40N       226277.0      2
X-MC       XY25       210671.0      2
H-MC       XY24       210310.0      1
X-LC       XY24       204165.0      2
H-MC       AR         185303.0      2
M-SC       XR         178569.0      2
X-SC       X40        173383.0      2
M-LC       XR         142976.0      1
X-LC       XY25       128171.0      2
M-SC       AR         123009.0      2
L-SC       OX40N       89836.0      1
           XR          89129.0      1
H-LC       AR          76146.0      1
M-MC       XY25        54733.0      1
L-MC       XR          50235.0      1
L-LC       XY25        28249.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 49.0 seconds
